# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.pipeline import make_pipeline

### 데이터 셋 읽어오기

In [ ]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [ ]:
df_train.head(10) # 학습용 데이터 살펴보기

In [ ]:
df_test.head() # 테스트 데이터 살펴보기

## 2. 데이터 전처리

### 레이블 인코딩

In [ ]:
df_train = df_train.drop_duplicates()
# Assuming df_train is your DataFrame
# Replace missing values in 'customer_country' with the mode of each 'lead_owner' group
mode_fill = df_train.groupby('lead_owner')['customer_country'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df_train['customer_country'] = df_train['customer_country'].fillna(mode_fill)

mode_fill = df_train.groupby('lead_owner')['customer_country.1'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df_train['customer_country.1'] = df_train['customer_country.1'].fillna(mode_fill)
# This code will fill the missing values in 'customer_country' with the mode of the corresponding 'lead_owner' group.
# If there is no mode (i.e., all values are missing) for a specific 'lead_owner', it will remain missing.

# Example:
# Before:
# df_train:
#    lead_owner  customer_country
# 0      owner1               USA
# 1      owner2               NaN
# 2      owner1               NaN
# 3      owner2               NaN

# After:
# df_train:
#    lead_owner  customer_country
# 0      owner1               USA
# 1      owner2               USA
# 2      owner1               USA
# 3      owner2               NaN  # If all values for owner2 are missing, it remains missing

# Count the number of missing values in 'customer_country'
missing_values_count = df_train['customer_country'].isnull().sum()

print("Number of missing values in customer_country:", missing_values_count)

missing_values_count = df_train['customer_country.1'].isnull().sum()

print("Number of missing values in customer_country.1:", missing_values_count)

# df_train.to_csv('train_customer_country_filled.csv', index=False)

In [ ]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [ ]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

print(df_train.isna().sum()/len(df_train))

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

다시 학습 데이터와 제출 데이터를 분리합니다.

In [ ]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]



### 2-2. 학습, 검증 데이터 분리

In [ ]:
df_train.fillna(df_train.mean())

In [ ]:
df_train.head()

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

## 3. 모델 학습

### 모델 정의 

In [ ]:
#model = DecisionTreeClassifier()
model = RandomForestClassifier()

In [ ]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))


In [ ]:
model.fit(x_train.fillna(0), y_train)

In [ ]:
pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

### GridSearchCV를 이용한 하이퍼파라미터 튜닝

In [ ]:
# # 탐색할 하이퍼파라미터 설정
# param_grid = {
#         'criterion': ['gini', 'entropy'],
#         'max_depth': [None, 5, 10, 15],
#         'min_samples_split': [2, 5, 10],
#         'min_samples_leaf': [1, 2, 4]
#     }
#
# # GridSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
# grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
# grid_search.fit(x_train.fillna(0), y_train)
#
# # 최적의 하이퍼파라미터 출력
# print('최고 평균 정확도 수치 : {0:.4f}'.format(grid_search.best_score_))
# print("Best Hyperparameters:", grid_search.best_params_)

KeyboardInterrupt: 

In [ ]:
# model = DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_leaf=4, min_samples_split=5)

In [ ]:
pipe = make_pipeline(
    RandomForestClassifier( class_weight= 'balanced')
)

# 최적값을 구하고 싶은 파라미터를 정리
dist = {
    'randomforestclassifier__max_depth' : [3,5,10,15],
    'randomforestclassifier__max_features' : [3,5,10],
    'randomforestclassifier__n_estimators' : [80, 100, 150, 200]
}

# RandomizedSearchCV
clf = RandomizedSearchCV(
		pipe,
        param_distributions = dist,
        n_iter = 10,  #위 조합 중 몇 개의 조합을 추출할 것인가?
		cv = 3,
        scoring = 'f1',  # 분류문제이므로
        verbose = 1,  # 진행상황 확인 여부
        n_jobs=-1)

clf.fit(x_train.fillna(0), y_train)
# 이렇게 실행하면, 10*3 = 30개의 fits 진행하게 된다. 시간이 꽤오래걸림


# 최적의 파라미터, 최고의 f1 score 찾는 방법
print('최적의 파라미터 :', clf.best_params_)
print('f1 score :', clf.best_score_)


# 각 조합으로 만들어진 모델을 순위별로 나열해서 확인해보는 방법
pd.DataFrame(clf.cv_results_).sort_values(by='rank_test_score').T


### 모델 학습

In [ ]:
# model.fit(x_train.fillna(0), y_train)
#
# rf_clf1 = RandomForestClassifier(n_estimators = 100,
#                                  max_depth = 12,
#                                  random_state = 0,)
# rf_clf1.fit(x_train.fillna(0), y_train)
# pred = rf_clf1.predict(x_val.fillna(0))
# print(classification_report(y_val, pred))

### 모델 성능 보기

In [ ]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [ ]:
# pred = model.predict(x_val.fillna(0))
# pred = rf_clf1.predict(x_val.fillna(0))
pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

In [ ]:
# model.fit(x_train.fillna(0),y_train)

## 4. 제출하기

In [ ]:
# # 최적의 모델로 예측
# best_dt_model = grid_search.best_estimator_
# y_pred = best_dt_model.predict(x_val.fillna(0))
#
# # 성능 평가
# print("\nClassification Report:")
# print(classification_report(y_val, y_pred))

### 테스트 데이터 예측

In [ ]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [ ]:
# test_pred = model.predict(x_test.fillna(0))
# test_pred = rf_clf1.predict(x_test.fillna(0))
test_pred = model.predict(x_test.fillna(0))

sum(test_pred) # True로 예측된 개수

### 제출 파일 작성

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**